# KHARMA Tutorial 
# CCA Numerical Series on Fluids and Plasmas (10/24/2024)
---
---

## PREREQUISITES

### Getting KHARMA

KHARMA can be found on Github [here](https://github.com/AFD-Illinois/kharma).
For this tutorial we'll be working with the `tutorial` branch which can be cloned like,

`git clone -b tutorial https://github.com/AFD-Illinois/kharma.git`

---

### Building pyharm

Additionally, we'll need `pyharm`, a Python package for analyzing GRMHD data products from HARM-based codes. The code is hosted [here](https://github.com/AFD-Illinois/pyharm), and can be obtained using git,

`git clone https://github.com/AFD-Illinois/pyharm.git`

It's generally good practice to isolate Python environments when working on different projects to avoid dependency hell. Also, the SCC at FI won't let us install python packages on the Python moduel provided by them. Let's download the python module (if you don't have it by default in your module stack), set up a virtual environment, and install `pyharm`,

```
module load python3
VENVDIR=PATH_WHERE_PYTHON_VENVS_ARE_STORED
python3 -m venv --system-site-packages $VENVDIR/pyharm
source $VENVDIR/pyharm/bin/activate
cd pyharm
pip3 install -e .
```
where, `VENVDIR` is the location where you would want to save your Python environments; for this tutorial we've created a virtual enviroment `pyharm`. More about Python virtual environments on Rusty [here](https://wiki.flatironinstitute.org/SCC/Software/PythonVirtualEnvironments). 

If you instead prefer working with `conda`, `pyharm` has a `environment.yml` file that can be used to set up an environment with `conda`. More about that [here](https://pyharm.readthedocs.io/en/latest/installing.html).

---

### Building KHARMA

KHARMA has two main dependencies which must be checked out before compiling it,
```
cd kharma
git submodule update --init --recursive
```

KHARMA has an extensive [wiki](https://github.com/AFD-Illinois/kharma/wiki) and lists the minimum requirements needed. Let us set up the module environment needed to compile KHARMA for GPUs on Rusty,

```
module load modules/2.3-20240529 slurm cmake gcc/11.4.0 cuda gsl openmpi hdf5 openblas
```
For CPUs, it's nearly the same, except we don't need the `cuda` module. 

KHARMA uses `cmake` to build the code called from the `make` shell script. This script in conjuction with machine-specific build files are help set up the module environment and pass necessary arguments to `cmake`. By default KHARMA parallelizes the build process, and therefore it is good practice to compile KHARMA on a compute node. To get an interactive session on a compute node on Rusty try,

`srun -N 1 --constraint=ib-icelake --pty bash -i`

To run KHARMA on NVIDIA A100s use the following command,

`./make.sh clean cuda a100`

The `cuda` argument tells Kokkos to build with CUDA, while the `a100` argument specifies the device architecture. KHARMA's build philosophy is to have separate machine-specific files that setup the relevant module environment and environment variables, e.g., there is a machine config file for Rusty (`machines/rusty.sh`) that builds for different architectures available on Rusty.

If you'd like to build for KHARMA to be run on CPU, say the icelake nodes, the build command would be,

`./make.sh clean skx`.

At the end, you should see the following output on the screen that comfirms KHARMA was built successfully,

![gpu build](images/screenshot_gpu_build.png)

OR

![cpu build](images/screenshot_cpu_build.png)

depending on the type of build. The postfix (`.cuda`, `.host`) in the executable name specifies the architecture KHARMA was built for. More about compiling [here](https://github.com/AFD-Illinois/kharma/wiki/Building-KHARMA).

---

### Running KHARMA

KHARMA can either be run in broadly two ways,
- By directly calliing the executable: `./kharma.host -i PROBLEM_PARAMETER_FILE`. This works best when you are running on a personal computer or are on a compute node via an interactive job, and have manually set the module stack and required environment variables correctly.

- Invoking `run.sh` or a batch script: Often, we don't want to manually set the environment and want the runtime environment to be set automatically based on the compile-time options. This is where `run.sh` or a machine-specific batch script comes in.

For this tutorial we'll use the slurm batch scripts located in the `scripts/tutorial/batch` directory. This script sets the SLURM directives, sources the machine file to set the module environment, and submits the job.

#### Problem 1: Hot cylinder in pressure equilibrium

